In [4]:
import sys,os
import pandas as pd
import pandapower as pp
import pandapower.auxiliary as aux  # for pandapowerNet typing
pd.options.display.max_rows = 20
from glob import glob
from collections import defaultdict

# import module from enclosing directory
cdir = sys.path[0] + '/..'
if cdir not in sys.path:
    sys.path.append(cdir)
from util import timeIt, Ith

# get data directory (make if not there)
cwd = os.getcwd()
transnet_dir = cwd + "/../data/transnet/"
if not os.path.exists(transnet_dir):
    os.makedirs(transnet_dir)
# download data

# store a transnet data instance, which is created from reading two .csv files
class TransnetOut():
    def __init__(self, name='', linepath='', nodepath='', read=False):
        """ input paths are paths to .csv files downloaded from transnet """
        self.name = name
        self.linepath = linepath
        self.nodepath = nodepath
        self.dataIsProcessed = False
        if read:
            self.read_data()
    
    def read_data(self, linepath='', nodepath=''):
        """ read the data from the csv files into pandas.Dataframes """
        if not linepath: linepath = self.linepath
        if not nodepath: nodepath = self.nodepath
        self.lines = pd.read_csv(linepath)
        self.nodes = pd.read_csv(nodepath)
        self.dataIsProcessed = True

    def __str__(self) -> str:
        if self.dataIsProcessed:
            ret = f"{self.name} has {len(self.lines)} lines and {len(self.nodes)} nodes"
            return ret
        return f"{self.name} at linepath='{self.linepath.split('/')[-1]}' and nodepath='{self.nodepath.split('/')[-1]}'"

    def __repr__(self) -> str:
        return self.__str__()

# get transnet data from folder
transnet_data = defaultdict(TransnetOut)
for path in glob(transnet_dir + "*.csv"):
    name = path.split('/')[-1]
    realname = '_'.join(name.split('_')[:-1])  # like "california"
    if 'lines' in name:
        transnet_data[realname].name = realname
        transnet_data[realname].linepath = path
    elif 'nodes' in name:
        transnet_data[realname].name = realname
        transnet_data[realname].nodepath = path

for t in transnet_data.values():
    print(t)
    t.read_data()
    print(t)

t_cal = transnet_data['california']
n = t_cal.nodes
l = t_cal.lines

print("\n\nnodes")
for k in n.columns:
    if k in ('type','voltage','frequency'):
        print(k, ', '.join(map(str,n[k].unique())))
print(t_cal.nodes.head())
print("\n\nlines")
for k in l.columns:
    if k in ('type','voltage','cables','frequency', 'r_ohm_km'):
        print(k, ', '.join(map(str,l[k].unique())))
print(t_cal.lines.head())

def add_transnet_bus(net: aux.pandapowerNet, node: pd.Series|dict, separate_voltage=False) -> None:
    """ add a transnet substation to a pandapower network 
    TODO:
    if separate_voltage, substations with multiple voltages are realistically treated by:
     - creating one long busbar for the highest voltage
     - for each lower voltage, creating a new small bus and connect it to the higher voltage bus
       via a transformer
     - keep track of the equivalences using a dict of 
       {original_id: {highest: original_id, lower1: bus_id1, lower2: bus_id2, ...},
       {} ...}
    """
    voltage = node['voltage']  # string of voltages like "115000;12000"
    if separate_voltage: 
        pass
    else:
        if ';' in voltage:
            voltage = voltage.split(';')[0]
        voltage = int(voltage)
    pp.create_bus(net, index=node['n_id'], name=node['name'], 
                  vn_kv=node['voltage'], transnet_type=node['type'],
                  geodata=(node['latitude'], node['longitude']), type='b' )  #is busbar

def add_transnet_gen(net: aux.pandapowerNet, node: pd.Series, p_mw=10) -> None:
    pp.create_gen(net, node['n_id'], p_mw=p_mw, vm_pu=1.0, name=node['name'], 
                  slack=False, type='unkown', controllable=True, 
                  max_p_mw=p_mw, min_p_mw=0, max_q_mvar=0.01, min_q_mvar=0)


def transnet_to_pp(nodes: pd.DataFrame, lines: pd.DataFrame, separate_voltage=False) -> aux.pandapowerNet:
    """ convert transnet data to power plant data 
    - adds all nodes first
       - substations with more than one voltage are added as one bus with the highest voltage """
    net = pp.create_empty_network()
    
    # create all nodes
    for i, node in nodes.iterrows():
        add_transnet_bus(net, node, separate_voltage)
        if node['type'] == 'substation':
            pass
        elif node['type'] == 'generator':
            # just assume all generators are 10MW for now
            add_transnet_gen(net, node, 10)
        elif node['type'] == 'plant':
            # just assume all plants are 50MW for now
            add_transnet_gen(net, node, 50)
    
    #  add all transmission lines
    for i, line in lines.iterrows():
        # freq = line['frequency'] if line['frequency'] else 60.0
        # don't care about type = line | cable
        # don't care about freq = line['frequency'] if line['frequency'] else 60.0
        # don't care about operator
        # don't care about 'not_accurate'
        start = line['n_id_start']
        end = line['n_id_end']
        # add line with standard type to ting
        if net['bus'].loc[start, 'vn_kv'] == 0:
            net['bus'].loc[start,'vn_kv'] = line['voltage']
        if net['bus'].loc[end, 'vn_kv'] == 0:
            net['bus'].loc[end,'vn_kv'] = line['voltage']
        if separate_voltage:
            pass
        cables = line['cables'] if line['cables'] else 1
        std_type = "NAYY 4x150 SE"
        if line['r_ohm_km'] and line['x_ohm_km'] and line['c_nf_km']:
            typedata = {'r_ohm_per_km': line['r_ohm_km'], 'x_ohm_per_km': line['x_ohm_km'], 'c_nf_per_km': 0.0, 'max_i_ka': line['i_th_max_km']}
            std_type = f"line {line['l_id']}"
            pp.create_std_type(net, typedata, std_type)
        id = pp.create_line(net, from_bus=start, to_bus=end, length_km=line['length_m']/1000, std_type=std_type,
                            vn_kv=line['voltage']/1000, parallel=cables, name=line['name'], index=line['l_id'])
        
    for i, node in nodes.iterrows():
        if node['type'] == 'substation':
            add_substation_to_pp(net, node)
        elif node['type'] == 'generator':
            add_gen_to_pp(net, node)
        elif node['type'] == 'plant':
            print(f"load {node['name']} not implemented")


california at linepath='california_lines.csv' and nodepath='california_nodes.csv'
california has 361 lines and 336 nodes


nodes
type substation, generator, plant
voltage 115000;12000, 230000;115000;13200;12000, 115000;12000;7200, 0, 230000;115000, 115000;21000;12000;7200, 115000, 115000;34500, 230000;115000;12000;7200, 230000;115000;60000;12000, 115000;12000;6600, 115000;60000;13200, 138000, 230000;138000, 138000;34500, 138000;69000, 220000;66000;16000, 220000, 220000;66000;16340, 230000, 220000;66000;12000, 220000;66000;16000;12000, 500000;220000, 220000;66000, 220000;69000, 220000;69000;12000, 230000;12000, 23000;115000;60000;13200;12000;7200, 230000;69000, 230000;60000, 230000;138000;34500, 500000;230000, 500000
frequency nan, 60.0
        n_id   longitude   latitude        type                    voltage  \
0   35656716 -122.148605  37.710544  substation               115000;12000   
1   33108052 -122.161080  37.847902  substation  230000;115000;13200;12000   
2  540298861 -122.21

In [3]:
from pandapower.converter import from_cim as cim2pp

net = cim2pp.from_cim([transnet_dir+'california_cim.xml'])

Exception: The FullModel is not given in the XML tree.